<a href="https://colab.research.google.com/github/aneeshcheriank/cowin-tracking/blob/main/barrier_reef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import files
path = files.upload()


# create a kaggle folder|
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [13]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c tensorflow-great-barrier-reef 

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
100% 14.2G/14.2G [05:26<00:00, 80.2MB/s]
100% 14.2G/14.2G [05:26<00:00, 46.6MB/s]


In [23]:
import zipfile
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil
import cv2
import matplotlib.pyplot as plt

In [15]:
# Extract the zip file
def extract(zip_file, output_path):
  os.makedirs(output_path, exist_ok=True)
  with zipfile.ZipFile(zip_file) as z:
    z.extractall(output_path)

zip_file = 'tensorflow-great-barrier-reef.zip'    
output_path = './data/extract'

extract(zip_file, output_path)

# Data Processing
- format the data columns
  - annotations were string, changed it into list
- split the data
  - data is split into train and test

In [17]:
BASE_DIR = './data/extract'

def data_processing(df, data_type = 'train'):

  # create the folders
  image_folder = './data/yolo/image/{data_type}'
  label_folder = './data/yolo/label/{data_type}'

  for folder in (image_folder, label_folder):
    os.mkdirs(folder, exist_ok=True)

  # copy image and label file into folders
  for _, row in tqdm(df.iterrows(), len(df)):

    annotations = row['annotations']

    # taking only those rows with annotations
    if len(annotations) > 0:
      
      folder_id, file_id = row['image_id'].split('-')
      folder = f'video_{folder_id}'
      file = f'{file_id}.jpg'
    
      # images
      shutil.copy(
          os.path.join(BASE_DIR, f'train_images/{folder}/{file}'),
          os.path.join(image_folder, file)
      )

      # labels
      yolo_lables = []
      for annotation in annotations:
        x = annoatation['x']
        y = annoatation['y']
        w = annoatation['width']
        h = annoatation['height']

        x_center = x + w/2
        y_center = y+ h/2

        x_center /= 1280
        y_center /= 720
        w /= 1280
        h /= 720

      yolo_label.append([x_center, y_center, w, h])
      np.savetxt(
          os.path.join(label_folder, )
          yolo_label,

      )





if __name__ == "__main__":
  train = pd.read_csv(
      os.path.join(BASE_DIR, 'train.csv')
  ) 

  train['annotations'] = train['annotations'].apply(eval)

  train, test = train_test_split(
      train,
      test_size = 0.25,
      random_state=42,
      shuffle=True
  )

  print(train.shape)
  print(test.shape)

(17625, 6)
(5876, 6)


In [18]:
train.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
2154,0,8399,4077,339,0-4077,[]
11782,1,8503,6630,2802,1-6630,[]
22948,2,29859,10207,2435,2-10207,[]
22884,2,29859,10143,2371,2-10143,[]
18252,2,26651,4184,512,2-4184,[]


In [20]:
type(train.annotations[0])

list

In [24]:
img = cv2.imread('./data/extract/train_images/video_0/0.jpg')
img.shape

(720, 1280, 3)